<a href="https://colab.research.google.com/github/AmbiTyga/Task-Stylumia/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c stumbleupon

Saving kaggle.json to kaggle.json
  0% 0.00/3.45M [00:00<?, ?B/s]
100% 3.45M/3.45M [00:00<00:00, 56.5MB/s]
 87% 7.00M/8.02M [00:00<00:00, 70.6MB/s]
100% 8.02M/8.02M [00:00<00:00, 73.7MB/s]
 92% 145M/157M [00:01<00:00, 119MB/s]
100% 157M/157M [00:01<00:00, 116MB/s]
  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 21.6MB/s]


In [3]:
!unzip -q /content/raw_content.zip

In [27]:
!pip install boilerpy3 -q

In [28]:
!pip install Unidecode -q

     |████████████████████████████████| 245kB 4.2MB/s 


In [2]:
!pip install scrapy -q

     |████████████████████████████████| 245kB 5.8MB/s 
     |████████████████████████████████| 3.2MB 14.5MB/s 
     |████████████████████████████████| 3.2MB 44.8MB/s 
     |████████████████████████████████| 245kB 48.0MB/s 
     |████████████████████████████████| 3.1MB 47.0MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 


In [29]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from unidecode import unidecode

In [8]:
 with open('/content/raw_content/1', 'r') as f:
   html = f.read()

In [25]:
sel = BeautifulSoup(html)

In [31]:
from boilerpy3 import extractors

extractor = extractors.ArticleExtractor()

In [34]:
all = extractor.get_doc(html)

In [39]:
s = all.get_text(True,True)
a = extractor.get_content(html)

In [43]:
for tag in ['script', 'style']:
  for el in sel.find_all(tag):
    el.extract()

In [45]:
type(el)

bs4.element.Tag

In [46]:
el.extract?